In [271]:
from analysis import collectConnections, print_with_stats
import pandas as pd
import re

In [272]:
to_namedtuple = lambda x: list(x.itertuples())
actions = to_namedtuple(pd.read_csv('../data/all-history-actions.csv'))
actions_df = pd.read_csv('../data/all-history-actions.csv')
findBillNum = re.compile('([HS]\d{1,5})')

In [273]:
actions_df

,id,action,branch,date
0,H1,Referred to the committee on House Ways and Means,House,2021-02-25T16:46:19.933Z
1,H1,"Reported, in part, by H4000",House,2021-04-15T04:00:00.000Z
2,H10,Referred to the committee on Public Service,House,2021-04-13T15:59:37.223Z
3,H10,Senate concurred,Senate,2021-04-13T15:59:37.223Z
4,H10,Hearing scheduled for 07/28/2021 from 01:00 PM...,Joint,2021-07-22T14:53:10.290Z
...,...,...,...,...
45467,s2729,Recommended new text for H4126,Senate,2022-02-28T05:00:00.000Z
45468,s2729,Rules suspended,Senate,2022-02-28T05:00:00.000Z
45469,s2729,Substituted as a new text for H4126,Senate,2022-02-28T05:00:00.000Z
45470,s2729,See H4126,Senate,2022-02-28T05:00:00.000Z


In [270]:
# TODO: parse action types: "referred to", "hearing scheduled", "reported from"
# TODO: create parse function that converts a raw action string to an array of dicts. Each dict has a 'type' field that identifies its type, and additional attribute fields

referred = re.compile(
    "(?P<type>referred to the committee) on (?P<committee_name>.*)", flags=re.IGNORECASE
)

hearing_scheduled = re.compile(
    "(?P<type>hearing scheduled) for (?P<date>.*) from (?P<time>.*) in (?P<forum>.*)", flags = re.IGNORECASE
)

reported_from = re.compile(
    "(?P<type>reported) from (?P<committee_name>.*)", flags= re.IGNORECASE
)

reading = re.compile(
    "(?P<type>read) (?P<reading>second|third|second and ordered to a third|)", flags= re.IGNORECASE
)

enacted = re.compile(
    "(?P<type>enacted) and (?P<info>.*)", flags= re.IGNORECASE
)

enacted_votes = re.compile(
    "(?P<type>enacted) - (?P<info>.*)", flags= re.IGNORECASE
)


action = actions[0].action
r = referred.match(action)

# actions[0] is a namedtuple. You can convert it to a dict with actions[0]._asdict()
# display(actions[0])
# print(action, r.groupdict())
# print(actions[0]._asdict().keys())

# def parse(action):
#     return [{"type": "reported"}, {"type": "referred", "committee_name": "Technology"}]

SyntaxError: invalid syntax (2545585676.py, line 20)

In [ ]:

action_types = [reading]

#creates list of actions from dictionaries
def extract_actions(actions):
    action_dict = action_to_dict(actions)
    extracted_actions = [action['action'] for action in action_dict]
    return extracted_actions


#converts list named tuples to a list of dictionaries
def action_to_dict(actions):
    action_dict = [action._asdict() for action in actions]
    return action_dict


#creates list of dictionaries of matching action types
def parse_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_actions.append(selected_action.groupdict())
    return selected_actions


def strip_committee(action_dict):
    cleaned_actions = []
    for dict_ in action_dict:
        cleaned_committee = dict_['committee_name'].strip()
        cleaned_dict = {'type': (dict_['type']), 'committee_name': (cleaned_committee)}
        cleaned_actions.append(cleaned_dict)
    return cleaned_actions


def append_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'info': selected_action_dict['info']}
                    selected_actions.append(selected_dict)
    return selected_actions


In [ ]:
action_dict = parse_actions(action_types, action_to_dict(actions))

action_dict


In [ ]:
appended_action_types = [enacted, enacted_votes]

append_actions(appended_action_types, action_to_dict(actions))

In [ ]:
bill_number = 'H4461'
connections = collectConnections(bill_number)
print_with_stats(connections)